# 토양 & 와인 데이터 전처리
1. Google Maps를 통해 Reverse Geocode로 지역명 확인  
2. 포도 품종 이름을 International Name으로 변환
    (i.e. Garnacha > Grenache, Pinot Nero > Pinot Noir 등)
3. Top10_Grape 생성하여 주요 품종은 1, 비주류 품종은 0으로 파생 변수 생성


### 0. 와인 데이터 불러오기

In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [1]:
df = pd.read_csv('./Wine Data/wine_soil_data.csv')
df

,Unnamed: 0,Longitude,Latitude,Soil Mapping Unit,Continent,Country,Region Lv1,Grape_Variety,SHARE_x,WRB4_x,...,CEC_CLAY,CEC_EFF,TEB,BSAT,ALUM_SAT,ESP,TCARBON_EQ,GYPSUM,ELEC_COND,Top10_Grape
0,0,-63.358333,-40.766667,12245,Americas,Argentina,Río Negro Province,Malbec,60,FLeu,...,63,22,22,94,0,4,2.0,0.7,2,NaN
1,1,-63.358333,-40.775000,12245,Americas,Argentina,Río Negro Province,Malbec,60,FLeu,...,63,22,22,94,0,4,2.0,0.7,2,NaN
2,2,-67.641667,-39.066667,12325,Americas,Argentina,Río Negro Province,Malbec,50,RGeu,...,75,20,20,98,0,3,1.8,4.0,1,NaN
3,3,-67.641667,-39.075000,12325,Americas,Argentina,Río Negro Province,Malbec,50,RGeu,...,75,20,20,98,0,3,1.8,4.0,1,NaN
4,4,-67.333333,-39.033333,12325,Americas,Argentina,Río Negro Province,Malbec,50,RGeu,...,75,20,20,98,0,3,1.8,4.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,2501,170.633333,-44.850000,6381,Oceania,New Zealand,Waitaki Valley (GI),Pinot Noir,60,LVgl,...,44,10,9,73,9,2,0.0,0.1,0,NaN
2502,2502,144.225000,-38.000000,5974,Oceania,Australia,Port Phillip,Syrah,40,LVcc,...,64,21,21,99,0,2,9.9,0.1,1,NaN
2503,2503,144.233333,-38.000000,5974,Oceania,Australia,Port Phillip,Syrah,40,LVcc,...,64,21,21,99,0,2,9.9,0.1,1,NaN
2504,2504,144.225000,-38.000000,5974,Oceania,Australia,Port Phillip,Pinot Noir,40,LVcc,...,64,21,21,99,0,2,9.9,0.1,1,NaN


In [3]:
df.drop(df.columns[0], axis = 1, inplace = True)
df

,Longitude,Latitude,Soil Mapping Unit,Continent,Country,Region Lv1,Grape_Variety,SHARE_x,WRB4_x,KOPPEN,...,CEC_CLAY,CEC_EFF,TEB,BSAT,ALUM_SAT,ESP,TCARBON_EQ,GYPSUM,ELEC_COND,Top10_Grape
0,-63.358333,-40.766667,12245,Americas,Argentina,Río Negro Province,Malbec,60,FLeu,B,...,63,22,22,94,0,4,2.0,0.7,2,NaN
1,-63.358333,-40.775000,12245,Americas,Argentina,Río Negro Province,Malbec,60,FLeu,B,...,63,22,22,94,0,4,2.0,0.7,2,NaN
2,-67.641667,-39.066667,12325,Americas,Argentina,Río Negro Province,Malbec,50,RGeu,B,...,75,20,20,98,0,3,1.8,4.0,1,NaN
3,-67.641667,-39.075000,12325,Americas,Argentina,Río Negro Province,Malbec,50,RGeu,B,...,75,20,20,98,0,3,1.8,4.0,1,NaN
4,-67.333333,-39.033333,12325,Americas,Argentina,Río Negro Province,Malbec,50,RGeu,B,...,75,20,20,98,0,3,1.8,4.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,170.633333,-44.850000,6381,Oceania,New Zealand,Waitaki Valley (GI),Pinot Noir,60,LVgl,C,...,44,10,9,73,9,2,0.0,0.1,0,NaN
2502,144.225000,-38.000000,5974,Oceania,Australia,Port Phillip,Syrah,40,LVcc,B,...,64,21,21,99,0,2,9.9,0.1,1,NaN
2503,144.233333,-38.000000,5974,Oceania,Australia,Port Phillip,Syrah,40,LVcc,B,...,64,21,21,99,0,2,9.9,0.1,1,NaN
2504,144.225000,-38.000000,5974,Oceania,Australia,Port Phillip,Pinot Noir,40,LVcc,B,...,64,21,21,99,0,2,9.9,0.1,1,NaN


### 1. Google Maps를 통해 Reverse Geocode로 지역명 확인 (실패..)

In [4]:
import googlemaps

gmap_key = ""
gmaps = googlemaps.Client(key = gmap_key)

In [9]:
# Look up an address with reverse geocoding
test_val = df.iloc[:1, :6]
test_val

,Longitude,Latitude,Soil Mapping Unit,Continent,Country,Region Lv1
0,-63.358333,-40.766667,12245,Americas,Argentina,Río Negro Province


In [11]:
result = gmaps.reverse_geocode((test_val.Latitude, test_val.Longitude))
result

/Users/sarahpark/miniforge3/envs/ds_study/lib/python3.8/site-packages/googlemaps/convert.py:54: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return ("%f" % float(arg)).rstrip("0").rstrip(".")


[{'address_components': [{'long_name': '6JMR+8M',
    'short_name': '6JMR+8M',
    'types': ['plus_code']},
   {'long_name': 'San Javier',
    'short_name': 'San Javier',
    'types': ['locality', 'political']},
   {'long_name': 'Adolfo Alsina Department',
    'short_name': 'Adolfo Alsina Department',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Río Negro Province',
    'short_name': 'Río Negro Province',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Argentina',
    'short_name': 'AR',
    'types': ['country', 'political']}],
  'formatted_address': '6JMR+8M San Javier, Río Negro Province, Argentina',
  'geometry': {'bounds': {'northeast': {'lat': -40.766625,
     'lng': -63.35825000000001},
    'southwest': {'lat': -40.76674999999999, 'lng': -63.358375}},
   'location': {'lat': -40.766667, 'lng': -63.358333},
   'location_type': 'GEOMETRIC_CENTER',
   'viewport': {'northeast': {'lat': -40.7653385197085,
     'lng': -63.35

In [18]:
region = result[0]['formatted_address'].split(',')[-2].strip()

'Río Negro Province'

In [25]:
df['gmaps_address'] = np.NaN

for idx, row in tqdm_notebook(df.iterrows()):
    result = gmaps.reverse_geocode((row.Latitude, row.Longitude))

    if result:
        try:
            address_parts = result[0]['formatted_address'].split(',')
            if len(address_parts) >= 2:
                df.loc[idx, 'gmaps_address'] = address_parts[-2].strip()
            else:
                df.loc[idx, 'gmaps_address'] = address_parts[0].strip()
        except (IndexError, KeyError) as e:
            df.loc[idx, 'gmaps_address'] = np.NaN

df.head()

/var/folders/10/g_4wtq6d0k5gc_g_zkc4v8c00000gn/T/ipykernel_12509/803452031.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm_notebook(df.iterrows()):


0it [00:00, ?it/s]

,Longitude,Latitude,Soil Mapping Unit,Continent,Country,Region Lv1,Grape_Variety,SHARE_x,WRB4_x,KOPPEN,...,CEC_EFF,TEB,BSAT,ALUM_SAT,ESP,TCARBON_EQ,GYPSUM,ELEC_COND,Top10_Grape,gmaps_address
0,-63.358333,-40.766667,12245,Americas,Argentina,Río Negro Province,Malbec,60,FLeu,B,...,22,22,94,0,4,2.0,0.7,2,NaN,Río Negro Province
1,-63.358333,-40.775000,12245,Americas,Argentina,Río Negro Province,Malbec,60,FLeu,B,...,22,22,94,0,4,2.0,0.7,2,NaN,Río Negro
2,-67.641667,-39.066667,12325,Americas,Argentina,Río Negro Province,Malbec,50,RGeu,B,...,20,20,98,0,3,1.8,4.0,1,NaN,Río Negro Province
3,-67.641667,-39.075000,12325,Americas,Argentina,Río Negro Province,Malbec,50,RGeu,B,...,20,20,98,0,3,1.8,4.0,1,NaN,Río Negro Province
4,-67.333333,-39.033333,12325,Americas,Argentina,Río Negro Province,Malbec,50,RGeu,B,...,20,20,98,0,3,1.8,4.0,1,NaN,Río Negro Province


In [71]:
df.drop(df.columns[-1], axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2506 entries, 0 to 2505
Data columns (total 51 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Longitude          2506 non-null   float64
 1   Latitude           2506 non-null   float64
 2   Soil Mapping Unit  2506 non-null   int64  
 3   Continent          2506 non-null   object 
 4   Country            2506 non-null   object 
 5   Region Lv1         2489 non-null   object 
 6   Grape_Variety      2501 non-null   object 
 7   SHARE_x            2506 non-null   int64  
 8   WRB4_x             2506 non-null   object 
 9   KOPPEN             2506 non-null   object 
 10  TEXTURE_USDA_x     2430 non-null   float64
 11  REF_BULK_DENSITY   2430 non-null   float64
 12  BULK_DENSITY       2506 non-null   float64
 13  DRAINAGE_x         2430 non-null   object 
 14  ROOT_DEPTH_x       2506 non-null   int64  
 15  AWC_x              2506 non-null   int64  
 16  ROOTS_x            724 n

### 2. 포도 품종 이름을 International Name으로 변환
    (i.e. Garnacha > Grenache, Pinot Nero > Pinot Noir 등)

In [36]:
len(df['Grape_Variety'].unique())

161

In [43]:
df['Grape_Variety'] = df['Grape_Variety'].apply(lambda x: x.lower().strip())
len(df['Grape_Variety'].unique())

133

In [44]:
grape_variety = [variety for variety in df['Grape_Variety'].unique()]
grape_variety = sorted(grape_variety)

In [45]:
grape_variety

['aglianico',
 'albarossa',
 'aleatico',
 'alicante bouschet',
 'aligote',
 'baga',
 'barbera',
 'bastardo',
 'bical',
 'blatina',
 'blaufrankisch',
 'bonarda',
 'bourboulenc',
 'brachetto',
 'brandy',
 'cabernet',
 'cabernet franc',
 'cabernet sauvignon',
 'cabernet savignon',
 'cabernet suavignon',
 'cardin',
 'carignan',
 'carignan gris',
 'carinena',
 'carmenere',
 'chambourcin',
 'chardonnay',
 'chenin blanc',
 'cinsault',
 'cinsaut',
 'clairette',
 'concord',
 'corvina',
 'counoise',
 'criolla grande',
 'croatina',
 'dolcetto',
 'feteasca neagra',
 'freisa',
 'galego dourado',
 'gamay',
 'garnacha',
 'gewurztraminer',
 'graciano',
 'grasevina',
 'greco nero',
 'grenache',
 'grenache blanc',
 'kadarka blau',
 'kratosija',
 'lagrein',
 'malbec',
 'marsanne',
 'marselan',
 'marzemino',
 'mauzac',
 'melon de bourgogne',
 'merlo',
 'merlot',
 'monastrell',
 'monastrelle',
 'montepulciano',
 'morellino',
 'mourvedr',
 'mourvedre',
 'mouvedre',
 'mtsvane',
 'muscadelle',
 'muscat',
 'mu

In [57]:
# cabernet은 모두 cabernet sauvignon으로 통일
# len(df[df['Grape_Variety'] == 'cabernet']) # 18개 

replace_val = [
    {'cabernet sauvignon': ['cabernet','cabernet savignon', 'cabernet suavignon']},
    {'cinsault': ['cinsaut']},
    {'merlot': ['merlo'] },
    {'monastrell': ['monastrelle']},
    {'mourvedre': ['mourvedr', 'mouvedre']},
    {'muscat': ['muscat hamburg'] },
    {'nielluccio':['niellucciu'] },
    {'petite verdot': ['petite verdot']},
    {'pinot gris': ['pinot grigio']},
    {'pinot noir': ['pinot nero', 'pino nero']},
    {'plavac_mali': ['plavac mali', 'plavac malii']},
    {'sciaccarello': ['sciaccarellu']},
    {'syrah': ['shiraz']}
]

# null값 처리
drop_val = ['pais (mission)','southern rhone red blend' ]


In [58]:
replace_dict = {}
for item in replace_val:
    for key, vals in item.items():
        for val in vals:
            replace_dict[val] = key

replace_dict   

{'cabernet': 'cabernet sauvignon',
 'cabernet savignon': 'cabernet sauvignon',
 'cabernet suavignon': 'cabernet sauvignon',
 'cinsaut': 'cinsault',
 'merlo': 'merlot',
 'monastrelle': 'monastrell',
 'mourvedr': 'mourvedre',
 'mouvedre': 'mourvedre',
 'muscat hamburg': 'muscat',
 'niellucciu': 'nielluccio',
 'petite verdot': 'petite verdot',
 'pinot grigio': 'pinot gris',
 'pinot nero': 'pinot noir',
 'pino nero': 'pinot noir',
 'plavac mali': 'plavac_mali',
 'plavac malii': 'plavac_mali',
 'sciaccarellu': 'sciaccarello',
 'shiraz': 'syrah'}

In [60]:
df['Grape_Variety'] = df['Grape_Variety'].replace(replace_dict)
df.loc[df['Grape_Variety'].isin(drop_val), 'Grape_Variety'] =  np.NaN

In [61]:
len(df['Grape_Variety'].unique())

116

### 3. Top10_Grape 생성하여 주요 품종은 1, 비주류 품종은 0으로 파생 변수 생성

주요 품종 (https://www.foodandwine.com/most-common-red-wine-7560943)
- Cabernet Sauvignon
- Merlot
- Pinot Noir
- Syrah
- Zinfandel
- Grenache
- Malbec
- Sangiovese
- Cabernet Franc
- Nebbiolo
- Tempranillo
- Montepulciano

In [64]:
print(df['Grape_Variety'].value_counts()[:20])

Grape_Variety
syrah                 420
cabernet sauvignon    352
pinot noir            296
merlot                209
cabernet franc        122
malbec                112
sangiovese            111
nebbiolo               99
grenache               84
petit verdot           48
barbera                44
corvina                36
carmenere              35
rondinella             35
tannat                 33
mourvedre              31
montepulciano          25
chardonnay             19
zinfandel              18
tempranillo            15
Name: count, dtype: int64


In [68]:
main_variety = ['cabernet sauvignon', 'merlot', 'pinot noir', 'syrah', 'zinfandel', 'grenache', 'malbec', 'sangiovese', 'cabernet franc', 'nebbiolo', 'tempranillo', 'montepulciano']

for idx, row in df.iterrows():
    if row['Grape_Variety'] in main_variety:
        df.loc[idx, 'Top10_Grape'] = 1
    else:
        df.loc[idx, 'Top10_Grape'] = 0

In [69]:
df['Top10_Grape'].value_counts()

Top10_Grape
1.0    1863
0.0     643
Name: count, dtype: int64

In [72]:
df.to_csv("./Wine Data/wine_soil_data_0717.csv")